

# initializing

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from datetime import datetime
import seaborn as sns

warnings.filterwarnings("ignore")

use_clean_data = True

# reading data

In [ ]:
if not use_clean_data :
    data = pd.read_csv('statistics.csv')
    col = pd.read_csv('sample.csv')
    data.columns=col.columns

# cleaning data

## reduce by business 

In [ ]:
if not use_clean_data :
    data['created_at']=data['created_at'].apply(lambda x:datetime.strptime(x[:7],'%Y-%m'))

    print('count of user created before 2017-06:')
    print(len(data[data['created_at'] < datetime.strptime('2017-06-01','%Y-%m-%d')]))
    data = data[data['created_at'] >= datetime.strptime('2017-06-01','%Y-%m-%d')]
    
    data.drop(columns=['followings_num',
                        'followers_num',
                        'needs_update',
                        'last_reward_app_version',
                        'last_wrong_reviews_count',
                        'last_correct_reviews_count',
                        'times_logged_in',
                        'invitation_count',
                        'min_weekly_rank',
                        'min_total_rank',
                        'max_weekly_score',
                        'max_total_score',
                        'max_coins',
                        'updated_at'],
              inplace=True,
              axis=1)

## checking noise

In [ ]:
data[data.games_played!=data.won_count+data.tie_count+data.lost_count]

In [ ]:
data.drop(data[data.games_played!=data.won_count+data.tie_count+data.lost_count].index,inplace=True)

In [ ]:
print(data.won_count.sum()/
      data.lost_count.sum())

In [ ]:
print(data.won_count_with_expiration.sum()/
      data.lost_count_with_expiration.sum())

In [ ]:
print(data.won_count_with_give_up.sum()/
      data.lost_count_with_give_up.sum())

In [ ]:
(data[data.purchased_amount>0].purchased_amount/data[data.purchased_amount>0].purchased_coins).hist()

In [ ]:
(data[data.times_spent_hint_freeze_time>0].coins_spent_on_hint_freeze_time
/data[data.times_spent_hint_freeze_time>0].times_spent_hint_freeze_time).hist(log=True)

In [ ]:
(data[data.times_spent_hint_delete_wrong>0].coins_spent_on_hint_delete_wrong
/data[data.times_spent_hint_delete_wrong>0].times_spent_hint_delete_wrong).hist(log=True)

In [ ]:
(data[data.times_spent_hint_answer_again>0].coins_spent_on_hint_answer_again
/data[data.times_spent_hint_answer_again>0].times_spent_hint_answer_again).hist(log=True)

In [ ]:
(data[data.times_spent_unlimited_games>0].coins_spent_on_unlimited_games
/data[data.times_spent_unlimited_games>0].times_spent_unlimited_games).hist(log=True)

In [ ]:
(data[data.times_changing_avatars>0].coins_spent_on_avatars
/data[data.times_changing_avatars>0].times_changing_avatars).hist(log=True)

In [ ]:
(data[data.times_spent_hint_answer_percentage>0].coins_spent_on_hint_answer_percentage
/data[data.times_spent_hint_answer_percentage>0].times_spent_hint_answer_percentage).hist(log=True)

In [ ]:
len(data[data.tournament_cups>data.tournaments_played])

In [ ]:
(data.purchased_coins-(data.coins_spent_on_avatars
                     +data.coins_spent_on_booster_package
                     +data.coins_spent_on_hint_answer_again
                     +data.coins_spent_on_hint_answer_percentage
                     +data.coins_spent_on_hint_delete_wrong
                     +data.coins_spent_on_hint_freeze_time
                     +data.coins_spent_on_unlimited_games)).describe()

## finding out and removing columns with unique value

In [ ]:
for c in ['accepted_questions_count','weekly_cups']:
    data[c].hist(log=True)
    plt.show()

In [ ]:
if not use_clean_data :
    unique_list=[]
    print('columns with unique value:')
    for i in range(len(data.columns)):
        if len(data.iloc[:,i].unique())==1:
            print(data.columns[i])
            unique_list+=[data.columns[i]]
    #my finding:
    unique_list+=['accepted_questions_count','weekly_cups']
    data.drop(columns=unique_list,inplace=True,axis=1)

## delete unactive player

In [ ]:
data = data[data.games_played>5]

In [ ]:
# player wihtout won or games is not active at all
'''
if not use_clean_data :
    for col in ['games_played','won_count']:
        fig,ax = plt.subplots()
        ax= data[col].hist()
        ax.set_title(col)
        side_text = plt.figtext(1,0.1,  str(data[col].describe()) 
                                        + '\n \n' 
                                        + str(data[col].value_counts().head())
                                , bbox=dict(facecolor='white'))
        fig.subplots_adjust(top=0.8)
        plt.show()

    # cleaning data
    data=data[data.games_played!=0]
    data=data[data.won_count!=0]
'''

## removing upper outlier with threshold : 5 / 10,000

In [ ]:
if not use_clean_data:
    for c in data.columns:
        if c not in ['created_at','updated_at']: 
            data = data[data[c]<data[c].quantile(.9995)] 

## cleaning negetive number

In [ ]:
if not use_clean_data:
    for c in data.columns:
        if c not in ['created_at','updated_at']:
            count_of_negetive = len(data[data[c]<0])
            if count_of_negetive>0:
                print(c,count_of_negetive)
                data.loc[data[c]<0,c]=0

# saving or loading data without noise

In [ ]:
if not use_clean_data :
    data.to_csv('statistics2.csv',index=False)
else:
    data= pd.read_csv('statistics2.csv')

# Defining goal

In [ ]:
# goal
data['has_bought'] = data.purchased_times>0
goal='has_bought'
data.has_bought.value_counts()

# Analysing data

In [ ]:
def plot_probability_per_numberic(data,col,goal,bin_size):
    df =  data.groupby(col).agg(['mean','count'])[goal]
    df.reset_index(inplace=True)
    j=bin_size
    l=[]
    left_bin=df.iloc[0][col]
    for i in range(len(df)):
        j-=df.iloc[i]['count']
        if j<0:
            left_bin=df.iloc[i][col]
            j=bin_size
        #df.iloc[i][col+'_bins']=left_bin
        l+=[left_bin]
    
    df[col+'_bins']=l
    df2 = (df.groupby(col+'_bins').agg('mean')['mean']).to_frame().reset_index()
    df2.reset_index(inplace=True)
    df2.rename(columns={'mean':'probability_of_'+goal},inplace=True)
    return plt.plot(df2[col+'_bins'].tolist(),df2['probability_of_'+goal].tolist())
#plot_probability_per_numberic(data,col,'has_bought',1000)

In [ ]:
def describe_field(data,col,goal,ignore = ['created_at','updated_at']):
    sample_fraction = .1
    bins_num = 30
    bins_size = 1000
    
    
    fig,axes = plt.subplots(1,3,figsize=(18,6))

    if col in ignore:        
        axes[0]= data[col].hist(bins=bins_num,ax=axes[0])

        
    else:
        axes[0].hist([data[(data[goal]==1)][col],
                  data[data[goal]!=1][col].sample(frac=sample_fraction, random_state=1)],
                 stacked=True,
                 bins=30,
                 label=['reached goal','losed goal']
                )
        
        sns.distplot(data[data[goal]][col],hist=True,kde=True,label='reached goal',ax=axes[1])
        sns.distplot(data[~data[goal]][col],hist=True,kde=True,label='losed goal', ax=axes[1])
        
        axes[2] = plot_probability_per_numberic(data,col,goal,bins_size)

        
    axes[1].set_title(col)
    side_text = plt.figtext(1,0.1,  str(data[col].describe()) 
                                    + '\n \n' 
                                    + str(data[col].value_counts().head())
                                    + '\n\n '+goal+':\n'
                                    + str(data[data[goal]][col].value_counts().head())
                            , bbox=dict(facecolor='white'))
    fig.subplots_adjust(top=0.8)
    plt.show()

In [ ]:
# 

In [ ]:
for i in range(1,len(data.columns)-1):
    print(i)
    col=data.columns[i]
    describe_field(data,col,goal)

In [ ]:
# 

In [ ]:
per_games_played_list=['won_count', 'tie_count', 'lost_count',
       'perfect_games', 
        'times_spent_hint_freeze_time','times_spent_hint_delete_wrong','times_spent_hint_answer_again',
                       'times_spent_unlimited_games','times_changing_avatars', 'times_spent_hint_answer_percentage',
                       'times_spent_booster_package',
        'won_count_with_expiration', 'won_count_with_give_up',
        'lost_count_with_expiration', 'lost_count_with_give_up',
        'times_viewing_adv_extra_game',
        'total_question_rating_count', 'question_up_rating_count', 'question_down_rating_count', 
                       'question_none_rating_count',
        'question_report_count', 
        'tournament_cups', 'tournaments_played',
        'number_of_invitation',
        'correct_answer_count']

data_per_game=data[per_games_played_list+['has_bought','games_played']]


for c in per_games_played_list:
    data_per_game[c]=data_per_game[c]/data_per_game.games_played
    describe_field(data_per_game,c,goal)


# spend coins

In [ ]:
data['times_spend_coins']= data.times_spent_hint_answer_again+data.times_spent_hint_answer_percentage+data.times_spent_hint_delete_wrong+data.times_spent_hint_freeze_time+data.times_spent_unlimited_games+data.times_spent_booster_package

In [ ]:
for i in range(1,len(data.columns)-1):
    print(i)
    col=data.columns[i]
    describe_field(data,col,goal='times_spend_coins')

In [ ]:
data[data.times_spend_coins<data.times_spend_coins.quantile(.95)].times_spend_coins.hist()

In [ ]:
data['times_spend_coins_per_game'] = data['times_spend_coins']/data.games_played

In [ ]:
data[data.times_spend_coins_per_game<data.times_spend_coins_per_game.quantile(.99)].times_spend_coins_per_game.hist()

In [ ]:

sns.jointplot(data=data, x='times_spend_coins', y='games_played', kind='reg', color='g')
